###IMPORTS

In [1]:
from osgeo import ogr, osr

import folium
import numpy as np
import pandas as pd
import sqlalchemy

from sqlalchemy import create_engine

###Build Common Functions for Processing Geodata

In [2]:
def build_json(s):
    ret_str = '{"type": "FeatureCollection","features": ['
    for ix, val in s.iteritems():
        ret_str = ret_str + val + ','
    return ret_str[:-1] + ']}'

###Setup Query Statements

In [3]:
sql_stmt_2014_new = """
    SELECT estimates_year, xref.tract_2010, ethnicity, ind, sum(val) as val FROM
        (SELECT [estimates_year],[mgra],[ethnicity]
           ,[popm_0to4],[popm_5to9],[popm_10to14],[popm_15to17],[popm_18to19]
           ,[popm_20to24],[popm_25to29],[popm_30to34],[popm_35to39],[popm_40to44]
           ,[popm_45to49],[popm_50to54],[popm_55to59],[popm_60to61],[popm_62to64]
           ,[popm_65to69],[popm_70to74],[popm_75to79],[popm_80to84],[popm_85plus]
           ,[popf_0to4],[popf_5to9],[popf_10to14],[popf_15to17],[popf_18to19]
           ,[popf_20to24],[popf_25to29],[popf_30to34],[popf_35to39],[popf_40to44]
           ,[popf_45to49],[popf_50to54],[popf_55to59],[popf_60to61],[popf_62to64]
           ,[popf_65to69],[popf_70to74],[popf_75to79],[popf_80to84],[popf_85plus]
         FROM [concep_test].[concep].[detailed_pop_tab_mgra]
         WHERE estimates_year = 2014 AND ethnicity in (1,2,3,4,5,6,7,8)) pvt
    UNPIVOT (val FOR ind IN (
           [popm_0to4],[popm_5to9],[popm_10to14],[popm_15to17],[popm_18to19]
           ,[popm_20to24],[popm_25to29],[popm_30to34],[popm_35to39],[popm_40to44]
           ,[popm_45to49],[popm_50to54],[popm_55to59],[popm_60to61],[popm_62to64]
           ,[popm_65to69],[popm_70to74],[popm_75to79],[popm_80to84],[popm_85plus]
           ,[popf_0to4],[popf_5to9],[popf_10to14],[popf_15to17],[popf_18to19]
           ,[popf_20to24],[popf_25to29],[popf_30to34],[popf_35to39],[popf_40to44]
           ,[popf_45to49],[popf_50to54],[popf_55to59],[popf_60to61],[popf_62to64]
           ,[popf_65to69],[popf_70to74],[popf_75to79],[popf_80to84],[popf_85plus])) as unpvt
    JOIN data_cafe.ref.vi_xref_geography_mgra_13 xref ON unpvt.mgra = xref.mgra_13
    GROUP BY
        estimates_year,xref.tract_2010,ethnicity,ind
"""

sql_stmt_2014_old = """
    SELECT estimates_year, xref.tract_2010, ethnicity, ind, sum(val) as val FROM
        (SELECT [estimates_year],[mgra],[ethnicity]
           ,[popm_0to4],[popm_5to9],[popm_10to14],[popm_15to17],[popm_18to19]
           ,[popm_20to24],[popm_25to29],[popm_30to34],[popm_35to39],[popm_40to44]
           ,[popm_45to49],[popm_50to54],[popm_55to59],[popm_60to61],[popm_62to64]
           ,[popm_65to69],[popm_70to74],[popm_75to79],[popm_80to84],[popm_85plus]
           ,[popf_0to4],[popf_5to9],[popf_10to14],[popf_15to17],[popf_18to19]
           ,[popf_20to24],[popf_25to29],[popf_30to34],[popf_35to39],[popf_40to44]
           ,[popf_45to49],[popf_50to54],[popf_55to59],[popf_60to61],[popf_62to64]
           ,[popf_65to69],[popf_70to74],[popf_75to79],[popf_80to84],[popf_85plus]
         FROM [estimates].[concep].[detailed_pop_tab_mgra]
         WHERE estimates_year = 2014 AND ethnicity in (1,2,3,4,5,6,7,8)) pvt
    UNPIVOT (val FOR ind IN (
           [popm_0to4],[popm_5to9],[popm_10to14],[popm_15to17],[popm_18to19]
           ,[popm_20to24],[popm_25to29],[popm_30to34],[popm_35to39],[popm_40to44]
           ,[popm_45to49],[popm_50to54],[popm_55to59],[popm_60to61],[popm_62to64]
           ,[popm_65to69],[popm_70to74],[popm_75to79],[popm_80to84],[popm_85plus]
           ,[popf_0to4],[popf_5to9],[popf_10to14],[popf_15to17],[popf_18to19]
           ,[popf_20to24],[popf_25to29],[popf_30to34],[popf_35to39],[popf_40to44]
           ,[popf_45to49],[popf_50to54],[popf_55to59],[popf_60to61],[popf_62to64]
           ,[popf_65to69],[popf_70to74],[popf_75to79],[popf_80to84],[popf_85plus])) as unpvt
    JOIN data_cafe.ref.vi_xref_geography_mgra_13 xref ON unpvt.mgra = xref.mgra_13
    GROUP BY
        estimates_year,xref.tract_2010,ethnicity,ind
"""

tract_sql = """
SELECT
  zone as tract_2010
  ,shape.STAsText() as wkt
FROM
  data_cafe.ref.geography_zone
WHERE
  geography_type_id = 59
"""

###Load Up the Data Frames

In [4]:
connection_string = 'mssql+pyodbc://sql2014a8/ws?driver=SQL+Server+Native+Client+11.0'

engine = create_engine(connection_string)

df_prelim = pd.read_sql(sql_stmt_2014_new, engine)
df_current = pd.read_sql(sql_stmt_2014_old, engine)
df_tract_shp = pd.read_sql(tract_sql, engine, index_col='tract_2010')

df_prelim.rename(columns={'val': 'val_prelim'}, inplace=True)
df_current.rename(columns={'val': 'val_current'}, inplace=True)

df_prelim.drop('estimates_year', axis=1, inplace=True)
df_current.drop('estimates_year', axis=1, inplace=True)

df_prelim.set_index(['tract_2010','ethnicity','ind'], inplace=True)
df_prelim.sortlevel(inplace=True)

df_current.set_index(['tract_2010','ethnicity','ind'], inplace=True)
df_current.sortlevel(inplace=True)

###Transform to Lat / Long and Build GeoJSON Representation of Each Record

In [18]:
source = osr.SpatialReference()
source.ImportFromEPSG(2230)

target = osr.SpatialReference()
target.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(source, target)

def create_GeoJson(row):
    geom_binary = ogr.CreateGeometryFromWkt(row.wkt)
    geom_binary.Transform(transform)
    geom_json = geom_binary.ExportToJson()
    
    properties = '"properties": {'
    
    for ix, val in row.iteritems():
        if ix != 'wkt' and ix != 'json':
            properties = properties + '"{0}": "{1}",'.format(ix, val)
    properties = properties[:-1] + '}'
    
    return '{{"type": "Feature", "id": "{0}", {1}, "geometry": {2}}}'.format(row.name, properties, geom_json)

###Process Data Frames for Change

In [6]:
compare_df = df_current.join(df_prelim)
compare_df['change'] = compare_df.val_prelim - compare_df.val_current
compare_df['pct_change'] = compare_df.val_prelim / compare_df.val_current - 1

compare_df.ix[compare_df['pct_change'] == np.inf, 'pct_change'] = np.NaN

In [7]:
tract_df = compare_df.groupby(level=0).sum().join(df_tract_shp)
tract_df['pct_change'] = tract_df['val_prelim'] / tract_df['val_current'] - 1
tract_df = tract_df.reindex(tract_df['pct_change'].abs().order(ascending=False).index)
tract_df[tract_df['pct_change'].abs() >= .01]

tract_df[['change','pct_change']].describe()

,change,pct_change
count,627.000000,626.000000
mean,-3.044657,-0.000618
std,20.868321,0.004443
min,-123.000000,-0.033273
25%,-10.000000,-0.002042
50%,-1.000000,-0.000205
75%,7.000000,0.001600
max,99.000000,0.020008


In [32]:
tract_df['popupContent'] = tract_df.apply(lambda x: 'Prelim Value: {0}<br/>Current Value: {1}'.format(x['val_prelim'], x['val_current']), axis=1)
tract_df['json'] = tract_df.apply(create_GeoJson, axis=1)

tract_df.ix[tract_df['pct_change'].abs() >= .01, 'json']

tract_2010
13103    {"type": "Feature", "id": "13103", "properties...
10601    {"type": "Feature", "id": "10601", "properties...
21000    {"type": "Feature", "id": "21000", "properties...
12501    {"type": "Feature", "id": "12501", "properties...
12502    {"type": "Feature", "id": "12502", "properties...
20904    {"type": "Feature", "id": "20904", "properties...
13302    {"type": "Feature", "id": "13302", "properties...
13203    {"type": "Feature", "id": "13203", "properties...
13312    {"type": "Feature", "id": "13312", "properties...
13418    {"type": "Feature", "id": "13418", "properties...
12600    {"type": "Feature", "id": "12600", "properties...
13104    {"type": "Feature", "id": "13104", "properties...
13205    {"type": "Feature", "id": "13205", "properties...
18300    {"type": "Feature", "id": "18300", "properties...
20028    {"type": "Feature", "id": "20028", "properties...
12900    {"type": "Feature", "id": "12900", "properties...
20025    {"type": "Feature", "id": "20025", "

In [33]:
build_json(tract_df.ix[tract_df['pct_change'].abs() >= .02, 'json'])

'{"type": "FeatureCollection","features": [{"type": "Feature", "id": "13103", "properties": {"val_current": "2765","val_prelim": "2673","change": "-92","pct_change": "-0.0332730560579","popupContent": "Prelim Value: 2673<br/>Current Value: 2765"}, "geometry": { "type": "Polygon", "coordinates": [ [ [ -117.08262899978899, 32.613483998759442 ], [ -117.08271900057251, 32.613713999311919 ], [ -117.08285599919488, 32.6140609984162 ], [ -117.08309800243305, 32.614668000389727 ], [ -117.08325900157396, 32.615085000112586 ], [ -117.0832660013334, 32.615101998873428 ], [ -117.08340099966101, 32.615451998561895 ], [ -117.08353500178096, 32.615791997621287 ], [ -117.08393799988539, 32.616817998231774 ], [ -117.08407299935008, 32.617159999152591 ], [ -117.08450500123524, 32.617038000969693 ], [ -117.08580099892376, 32.616674998689362 ], [ -117.08623399931868, 32.616553999083436 ], [ -117.08638399914211, 32.616511000435743 ], [ -117.08683700142851, 32.616382999934146 ], [ -117.08698800014118, 32.61

In [34]:
map_osm = folium.Map(location=[32.8, -117.4], tiles='CartoDB positron')

map_osm.geo_json(geo_str=build_json(tract_df.ix[tract_df['pct_change'].abs() >= .01, 'json']))

map_osm

In [ ]:
df